# Attention LSTM 모델

# 준비

In [19]:
!nvidia-smi

Wed Apr 21 03:57:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    36W / 250W |   8907MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
%cd /content/drive/MyDrive/Project_Music_KW

/content/drive/MyDrive/Project_Music_KW


# 패키지 임포트

In [22]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras import optimizers
from keras import models
from keras import layers
from keras.utils import np_utils
import keras.backend as K

# dataset 불러오기

라흐마니노프 피아노곡 http://www.piano-midi.de/rach.htm

In [23]:
# 하이퍼퍼라미터
embed_size = 256
rnn_units = 4096

# 데이터 이름
data_name = 'Rachmaninov'
pickle_name = 'Rachmaninov_pickle'

## 기존에 00_preprocessing.ipynb을 통해 악보를 정수로 변환한 pickle 파일을 불러옵니다.

In [24]:
with open('./preprocessing/data/{}_notes'.format(pickle_name), 'rb') as f:
  notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
with open('./preprocessing/data/{}_durations'.format(pickle_name), 'rb') as f:
  durations = pickle.load(f)

## 모델에서 사용할 수 있도록 전처리를 마무리합니다.

In [25]:
def get_distinct(elements):
    # Get all pitch names
    element_names = sorted(set(elements))
    n_elements = len(element_names)
    return (element_names, n_elements)

In [26]:
def create_lookups(element_names):
    # create dictionary to map notes and durations to integers
    element_to_int = dict((element, number) for number, element in enumerate(element_names))
    int_to_element = dict((number, element) for number, element in enumerate(element_names))

    return (element_to_int, int_to_element)

In [27]:
# 고유한 음표와 박자 얻어오기
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open('./preprocessing/data/{}_distincts'.format(pickle_name), 'wb') as f:
    pickle.dump(distincts, f)

# 음표와 박자 룩업 딕셔너리 만들고 저장하기
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open('./preprocessing/data/{}_lookups'.format(pickle_name), 'wb') as f:
    pickle.dump(lookups, f)

In [28]:
def prepare_sequences(notes, durations, lookups, distincts):
    """ Prepare the sequences used to train the Neural Network """
    seq_len = 100

    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups
    note_names, n_notes, duration_names, n_durations = distincts

    notes_network_input = []
    notes_network_output = []
    durations_network_input = []
    durations_network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - seq_len, 1):
        notes_sequence_in = notes[i:i + seq_len]
        notes_sequence_out = notes[i + seq_len]
        notes_network_input.append([note_to_int[char] for char in notes_sequence_in])
        notes_network_output.append(note_to_int[notes_sequence_out])

        durations_sequence_in = durations[i:i + seq_len]
        durations_sequence_out = durations[i + seq_len]
        durations_network_input.append([duration_to_int[char] for char in durations_sequence_in])
        durations_network_output.append(duration_to_int[durations_sequence_out])

    n_patterns = len(notes_network_input)

    # reshape the input into a format compatible with LSTM layers
    notes_network_input = np.reshape(notes_network_input, (n_patterns, seq_len))
    durations_network_input = np.reshape(durations_network_input, (n_patterns, seq_len))
    network_input = [notes_network_input, durations_network_input]

    notes_network_output = np_utils.to_categorical(notes_network_output, num_classes=n_notes)
    durations_network_output = np_utils.to_categorical(durations_network_output, num_classes=n_durations)
    network_output = [notes_network_output, durations_network_output]

    return (network_input, network_output)

In [29]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts)

In [30]:
network_input[0].shape, network_input[1].shape

((10083, 100), (10083, 100))

In [31]:
network_input[0][5,:]

array([2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164,
       2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164,
       2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164,
       2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164,
       2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164,
       2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164,
       2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164,
       2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164, 2164,
       2164, 2164, 2164, 2164, 2164, 2164, 2164,   16, 1836,  532, 1850,
        474])

In [32]:
network_output[0].shape, network_output[1].shape

((10083, 2165), (10083, 28))

## 모델링을 수행합니다.

In [33]:
notes_in = layers.Input(shape = (None, ), name = 'notes_in')
durations_in = layers.Input(shape = (None, ), name = 'durations_in')

x1 = layers.Embedding(n_notes, embed_size)(notes_in)
x2 = layers.Embedding(n_durations, embed_size)(durations_in)

x = layers.Concatenate()([x1, x2])

x = layers.LSTM(
    rnn_units,
    return_sequences = True,
)(x)

x = layers.LSTM(
    rnn_units,
    return_sequences = True
)(x)

e = layers.Dense(1, activation = 'tanh')(x)
e = layers.Reshape([-1])(e)
alpha = layers.Softmax()(e)

alpha_repeated = layers.RepeatVector(rnn_units)(alpha)

c = layers.Permute([2, 1])(alpha_repeated)
c = layers.Multiply()([x, c])
c = layers.Lambda(lambda xin: K.sum(xin, axis = 1), output_shape = (rnn_units,))(c)

notes_out = layers.Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
durations_out = layers.Dense(n_durations, activation = 'softmax', name = 'duration')(c)

model = models.Model([notes_in, durations_in], [notes_out, durations_out], name = 'model')
att_model = models.Model([notes_in, durations_in], alpha)

opti = optimizers.RMSprop(lr = 0.00000001)
model.compile(loss = ['categorical_crossentropy', 'categorical_crossentropy'],
              optimizer = opti)

In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
notes_in (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
durations_in (InputLayer)       [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    554240      notes_in[0][0]                   
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    7168        durations_in[0][0]               
______________________________________________________________________________________________

In [35]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os

weights_folder = './weights/Rebuild_LSTM/Rachmaninov'
model.load_weights(os.path.join(weights_folder, "weights.h5"))

checkpoint = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor = 'loss',
    verbose = 0,
    save_best_only = True,
    mode = 'min'
)

early_stopping = EarlyStopping(
    monitor = 'loss',
    restore_best_weights = True,
    patience = 10
)


callbacks_list = [checkpoint,
                 early_stopping]

In [ ]:
model.fit(network_input, network_output,
          epochs = 1000,
          batch_size = 16,
          validation_split = 0.2,
          callbacks = callbacks_list,
          shuffle = True)

Epoch 1/1000
505/505 [==============================] - ETA: 0s - loss: 0.2269 - pitch_loss: 0.1735 - duration_loss: 0.0534